In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
data = pd.read_csv("E:\Virtual_room\project\AIrecruiter\data\processed\processed_resume.csv")

In [3]:
data.head()

,ID,Resume_str,Category,drop_duplicate,cleaned_resume,lemmatize_data,tokenize_data
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR,"2004 shows heads collateral, treatments, Shrub...",shows heads collateral treatments shrub...,show head collateral treatment shrubel billing...,"['show', 'head', 'collateral', 'treatment', 's..."
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...",HR,"changes Background hire shows Maintained Pro, ...",changes background hire shows maintained pro ...,change background hire show maintained pro sto...,"['change', 'background', 'hire', 'show', 'main..."
2,33176873,HR DIRECTOR Summary Over 2...,HR,"changes Develped hire Earned: Education, Corel...",changes develped hire earned education corel...,change develped hire earned education corel mi...,"['change', 'develped', 'hire', 'earned', 'educ..."
3,27018550,HR SPECIALIST Summary Dedica...,HR,payroll areas. delivery process adhering medic...,payroll areas delivery process adhering medic...,payroll area delivery process adhering medical...,"['payroll', 'area', 'delivery', 'process', 'ad..."
4,17812897,HR MANAGER Skill Highlights ...,HR,"Advocacy hire decision one-on-one performance,...",advocacy hire decision one on one performance ...,advocacy hire decision one one performance fun...,"['advocacy', 'hire', 'decision', 'one', 'one',..."


In [4]:
#target 
target = pd.get_dummies(data['Category'])

In [6]:
target.columns.values

array(['ACCOUNTANT', 'ADVOCATE', 'AGRICULTURE', 'APPAREL', 'ARTS',
       'AUTOMOBILE', 'AVIATION', 'BANKING', 'BPO', 'BUSINESS-DEVELOPMENT',
       'CHEF', 'CONSTRUCTION', 'CONSULTANT', 'DESIGNER', 'DIGITAL-MEDIA',
       'ENGINEERING', 'FINANCE', 'FITNESS', 'HEALTHCARE', 'HR',
       'INFORMATION-TECHNOLOGY', 'PUBLIC-RELATIONS', 'SALES', 'TEACHER'],
      dtype=object)

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(data['lemmatize_data'],target, test_size=0.20, random_state=40, shuffle=True, stratify=target, train_size=0.8)

In [12]:
X_train = np.asarray(X_train).astype('str')
X_test = np.asarray(X_test).astype('str')

In [13]:
#create the text encoder
vocab_size=1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size
)
encoder.adapt(X_train)
vocab = encoder.get_vocabulary()

In [14]:
model = tf.keras.Sequential([
                            tf.keras.layers.Input(shape=(), name="text", dtype=tf.string),
                            encoder,
                            tf.keras.layers.Embedding(
                                input_dim=len(vocab),
                                output_dim=64,
                                # Use masking to handle the variable sequence lengths
                                mask_zero=True),
                            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
                            tf.keras.layers.Dense(128, activation='relu'),
                            tf.keras.layers.Dropout(0.5),
                            tf.keras.layers.Dense(256, activation='relu'),
                            tf.keras.layers.Dropout(0.5),
                            tf.keras.layers.Dense(256, activation='relu'),
                            tf.keras.layers.Dropout(0.5),
                            tf.keras.layers.Dense(256, activation='relu'),
                            tf.keras.layers.Dropout(0.5),
                            tf.keras.layers.Dense(128, activation='relu'),
                            tf.keras.layers.Dense(24, activation='softmax')
])

In [15]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [16]:
history = model.fit(X_train,y_train, epochs=20,
                    validation_data=(X_test,y_test),
                    validation_steps=30)

Epoch 1/20
63/63 [==============================] - 180s 3s/step - loss: 3.1683 - accuracy: 0.0393 - val_loss: 3.1517 - val_accuracy: 0.0483
Epoch 2/20
63/63 [==============================] - 167s 3s/step - loss: 3.1518 - accuracy: 0.0443 - val_loss: 3.1437 - val_accuracy: 0.0282
Epoch 3/20
63/63 [==============================] - 170s 3s/step - loss: 3.1253 - accuracy: 0.0488 - val_loss: 3.0399 - val_accuracy: 0.0483
Epoch 4/20
63/63 [==============================] - 180s 3s/step - loss: 3.0109 - accuracy: 0.0775 - val_loss: 2.9767 - val_accuracy: 0.0785
Epoch 5/20
63/63 [==============================] - 196s 3s/step - loss: 2.9522 - accuracy: 0.1087 - val_loss: 2.9383 - val_accuracy: 0.1147
Epoch 6/20
63/63 [==============================] - 211s 3s/step - loss: 2.8658 - accuracy: 0.1203 - val_loss: 2.8767 - val_accuracy: 0.1227
Epoch 7/20
63/63 [==============================] - 223s 4s/step - loss: 2.7821 - accuracy: 0.1223 - val_loss: 4.1044 - val_accuracy: 0.0584
Epoch 8/20
63